Dhruv Gandhi and Aayush Dhiman

In [1]:
# https://forums.trakt.tv/t/id-like-an-honest-opinion-which-rating-system-you-trust/1779/2

In [2]:
import os
client_id = os.environ.get('client_id')
client_secret = os.environ.get('client_secret')

In [3]:
# print(client_id)
# print(client_secret)

In [4]:
import pandas as pd
from urllib.request import Request, urlopen
import json
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

def request_trakt(url):
    headers = {
        "trakt-api-version": "2",
        "Content-Type": "application/json",
        "trakt-api-key": client_id
    }
    request = Request(url, headers=headers)
    try:
        response_body = urlopen(request).read()
        return response_body
    except Exception as e:
        if hasattr(e, 'code') and e.code == 429:
            time.sleep(5)
            return request_trakt(url)
        else:
            print("Error: ", e)
            return None
    
base_url = "https://api.trakt.tv"

In [5]:
# url = f"{base_url}/lists/trending"
pop_movies_df = pd.DataFrame()

for page in range(1, 15):
    url = f"{base_url}/movies/trending?page={page}"
    response = request_trakt(url)

    if response:
        data = json.loads(response)
        pop_movies_df = pd.concat([pop_movies_df, pd.DataFrame(data)], ignore_index=True)

In [6]:
pop_movies_df

,watchers,movie
0,3168,"{'title': 'Mickey 17', 'year': 2025, 'ids': {'trakt': 540151, 'slug': 'mickey-17-2025', 'imdb': 'tt12299608', 'tmdb': 696506}}"
1,2331,"{'title': 'Black Bag', 'year': 2025, 'ids': {'trakt': 997729, 'slug': 'black-bag-2025', 'imdb': 'tt30988739', 'tmdb': 1233575}}"
2,2313,"{'title': 'A Minecraft Movie', 'year': 2025, 'ids': {'trakt': 763995, 'slug': 'a-minecraft-movie-2025', 'imdb': 'tt3566834', 'tmdb': 950387}}"
3,1986,"{'title': 'A Working Man', 'year': 2025, 'ids': {'trakt': 966319, 'slug': 'a-working-man-2025', 'imdb': 'tt9150192', 'tmdb': 1197306}}"
4,1038,"{'title': 'The Monkey', 'year': 2025, 'ids': {'trakt': 904978, 'slug': 'the-monkey-2025', 'imdb': 'tt27714946', 'tmdb': 1124620}}"
...,...,...
135,87,"{'title': 'Back to the Future', 'year': 1985, 'ids': {'trakt': 74, 'slug': 'back-to-the-future-1985', 'imdb': 'tt0088763', 'tmdb': 105}}"
136,87,"{'title': 'Tangled', 'year': 2010, 'ids': {'trakt': 25084, 'slug': 'tangled-2010', 'imdb': 'tt0398286', 'tmdb': 38757}}"
137,86,"{'title': 'Iron Man 2', 'year': 2010, 'ids': {'trakt': 5355, 'slug': 'iron-man-2-2010', 'imdb': 'tt1228705', 'tmdb': 10138}}"
138,86,"{'title': 'Harry Potter and the Order of the Phoenix', 'year': 2007, 'ids': {'trakt': 549, 'slug': 'harry-potter-and-the-order-of-the-phoenix-2007', 'imdb': 'tt0373889', 'tmdb': 675}}"


In [7]:
movie_id_list = []
username_set = set()
for i in range(len(pop_movies_df)):
    # print(pop_movies_df.loc[i]['title'])
    movie_info = pop_movies_df.loc[i]['movie']['ids']['trakt']
    movie_id_list.append(movie_info)
print(movie_id_list)
# df.loc[0]['ids']['trakt']

[540151, 997729, 763995, 966319, 904978, 916302, 964679, 1017204, 614652, 655031, 156893, 871757, 763780, 781338, 600937, 905958, 1005024, 856080, 905982, 994270, 970418, 656016, 495543, 1032845, 753684, 290190, 365073, 102156, 977993, 409281, 748546, 82405, 190430, 808857, 391372, 395672, 449452, 685641, 955488, 611948, 120, 929147, 6982, 191797, 1043205, 481, 88, 191798, 730642, 293904, 149640, 16662, 990784, 297485, 14701, 349425, 175475, 399330, 907654, 545, 51342, 1041018, 783046, 918315, 12269, 1157, 820687, 77349, 537449, 367444, 181256, 783020, 75735, 611574, 181311, 90, 203309, 815128, 882689, 759944, 822464, 546, 287071, 276, 651, 957456, 89, 169105, 12601, 990700, 183371, 561219, 432, 1416, 234, 196213, 5853, 916704, 698292, 965558, 922793, 547, 1014914, 888803, 1159813, 1134823, 210803, 9, 474180, 72719, 789, 322988, 977523, 1027240, 610, 1012109, 548, 10322, 635041, 228, 72297, 18, 1062255, 7, 4242, 167397, 762, 866348, 798786, 1071852, 34073, 205127, 927609, 50131, 71938,

In [8]:
for i in range(len(movie_id_list)):
    # print(movie_id_list[i])
    url = f"{base_url}/movies/{movie_id_list[i]}/watching"
    response = request_trakt(url)
    if response:
        data = json.loads(response)
        watching_movie_df = pd.DataFrame(data)
        for i in range(len(watching_movie_df)):
            # print(watching_movie_df.loc[i]['username'])
            formatted_username = watching_movie_df.loc[i]['username'].replace(' ', '-')
            username_set.add(formatted_username)
        # display(df)

In [9]:
username_set

{'rhuanbello',
 'kmatejka',
 'yuki1121',
 'wildlifechoruspt',
 'bigarod',
 'tuffythejeep1',
 '1983ron',
 'ZombieManki',
 'rycook123',
 'hurricane090',
 'TYTS',
 'paco_pacp',
 'Akiles77',
 'kreuzgriff',
 'LuisM09008',
 'aaron2gh',
 'Jalitoo23',
 'Jozelitoyou',
 'jnjperera',
 'smiro',
 'morty2000',
 'pmihojevic',
 'Barkley135',
 'Parrahandy',
 'Flaviodo',
 'JuJu473',
 'vonson',
 'fatccimex54',
 'grimfor',
 'TarjetaHumana',
 'ottom94',
 'Orlandols',
 'Nonoche',
 'sandman564',
 'RageLXIV',
 'dilipthekoala',
 'cawis',
 'Vorots1',
 'purplemc',
 'Ngranju',
 'speedygonzalez',
 'kerevizz',
 'louhel',
 'Flash055',
 'a17lancer',
 'fuzzypickles',
 'KentishGambino',
 'andy_t69',
 'PaloteMontblanc',
 '3ACEBC90',
 'pacotron',
 'espensgr',
 'gary16jan',
 'alpbost',
 'gon9gon',
 'kbarbary',
 'theonekaneda',
 'Delkz',
 'gst.vo',
 'paullson',
 'cainsblade',
 'bukhalo',
 'miriamYmarina',
 'timbuhr',
 'tlobrochill',
 'MitchFMitchF',
 'ssoprano',
 'oggedogge',
 'Ydnaback',
 'martineasthope',
 'Maximustrokol

In [10]:
# url = f"{base_url}/users/dhruvg85/lists/in-progress/items"
overall_movie_ratings_df = pd.DataFrame(columns=['username', 'movie_id', 'rating'])

for username in username_set:
    # print(username)
    url = f"{base_url}/users/{username}/ratings"
    response = request_trakt(url)
    if response:
        data = json.loads(response)
        ratings_df = pd.DataFrame(data)
        if len(ratings_df) > 0:
            if 'type' in ratings_df.columns:
                ratings_movie_df = ratings_df[ratings_df['type'] == 'movie']
                # display(ratings_movie_df)
                if len(ratings_movie_df) > 0:
                    ratings_movie_df = ratings_movie_df[['rating', 'movie']].reset_index(drop=True)
                # display(ratings_movie_df)
                user_df = pd.DataFrame()
                for i in range(len(ratings_movie_df)):
                    movie_info = ratings_movie_df.loc[i]['movie']
                    movie_id = movie_info['ids']['trakt']
                    rating = ratings_movie_df.loc[i]['rating']
                    
                    user_df = pd.concat([user_df, pd.DataFrame({'username': [username], 
                                                                'movie_id': [movie_id], 
                                                                'rating': [rating]})], ignore_index=True)
                overall_movie_ratings_df = pd.concat([overall_movie_ratings_df, user_df], ignore_index=True)


Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Unauthorized
Error:  HTTP Error 405: Method Not Allowed
Error:  HTTP Error 401: Una

In [11]:
overall_movie_ratings_df2 = overall_movie_ratings_df.drop_duplicates(subset=['username', 'movie_id'])
pivoted = overall_movie_ratings_df2.pivot(index='username', columns='movie_id', values='rating')        
pivoted.to_csv('trakt_movie_ratings.csv')

In [12]:
len(movie_id_list)
movie_id_list[0]

540151

In [13]:
movie_posters_df = pd.DataFrame(columns=['movie_id', 'poster'])
for i in range(len(movie_id_list)):
    # print(movie_id_list[i])
    url = f"{base_url}/movies/{movie_id_list[i]}?extended=images"
    response = request_trakt(url)
    if response:
        data = json.loads(response)
        movie_df = pd.DataFrame(data)
        # display(movie_df)
        if len(movie_df) > 0:
            if 'images' in movie_df.columns:
                movie_image_url = movie_df.loc['poster', 'images']
                # print(movie_image_url[0])
                movie_posters_df = pd.concat([movie_posters_df, pd.DataFrame({'movie_id': [movie_id_list[i]],
                                                                        'poster': f'https://{movie_image_url[0]}'})], ignore_index=True)
                
movie_posters_df.to_csv('trakt_movie_posters.csv')
# movie_image_url = f"{base_url}/movies/{movie_id_list[0]}?extended=images"

In [14]:
movie_posters_df

,movie_id,poster
0,540151,https://walter-r2.trakt.tv/images/movies/000/540/151/posters/thumb/748e123b39.jpg.webp
1,997729,https://walter-r2.trakt.tv/images/movies/000/997/729/posters/thumb/f935858c9f.jpg.webp
2,763995,https://walter-r2.trakt.tv/images/movies/000/763/995/posters/thumb/37c2ee9e9d.jpg.webp
3,966319,https://walter-r2.trakt.tv/images/movies/000/966/319/posters/thumb/e7aec3b005.jpg.webp
4,904978,https://walter-r2.trakt.tv/images/movies/000/904/978/posters/thumb/8c59f71b15.jpg.webp
...,...,...
135,74,https://walter-r2.trakt.tv/images/movies/000/000/074/posters/thumb/c608400a0d.jpg.webp
136,25084,https://walter-r2.trakt.tv/images/movies/000/025/084/posters/thumb/c630b81726.jpg.webp
137,5355,https://walter-r2.trakt.tv/images/movies/000/005/355/posters/thumb/dabd6f1e13.jpg.webp
138,549,https://walter-r2.trakt.tv/images/movies/000/000/549/posters/thumb/919cf5c76e.jpg.webp


In [17]:
movie_poster_storage = 'movie_posters'
if not os.path.exists(movie_poster_storage):
    os.makedirs(movie_poster_storage)
# Please cache all images! All images are required to be cached in your app or server and not loaded directly from our CDN. 
# Hotlinking images is not allowed and will be blocked.
for i in range(len(movie_posters_df)):
    movie_id = movie_posters_df.loc[i]['movie_id']
    poster_url = movie_posters_df.loc[i]['poster']
    poster_filename = os.path.join(movie_poster_storage, f"{movie_id}.jpg")
    if not os.path.exists(poster_filename):
        try:
            req = Request(poster_url, headers={'User-Agent': 'Mozilla/5.0'})
            with urlopen(req) as response, open(poster_filename, 'wb') as out_file:
                out_file.write(response.read())
        except Exception as e:
            print(f"Error downloading {poster_url}: {e}")
    else:
        print(f"File {poster_filename} already exists, skipping download.")
    # break

In [ ]:
# https://walter-r2.trakt.tv/images/movies/000/540/151/posters/thumb/748e123b39.jpg.webp


In [ ]:
# /search/trakt/:id?id_type=movie